In [12]:
import torch
print("Colab GPU available:", torch.cuda.is_available())
# !nvidia-smi

Colab GPU available: True


In [13]:
!nvidia-smi

Fri Nov 28 04:50:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
!ls

In [19]:
!cmake --version

cmake version 3.31.10

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [28]:
!rm -rf ./*

In [29]:
!ls

# Make a new clone from the git repo and then run it

In [30]:
# Always start from /content
%cd /content

# Remove any old copies so paths don't get nested
!rm -rf GPU_mode

# Clone your repo (note the exact repo name in the URL)
!git clone https://github.com/parthshinde1221/GPU_mode.git

# Enter the repo root
%cd GPU_mode

# Sanity check: you MUST see CMakeLists.txt here
!ls


/content
Cloning into 'GPU_mode'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 22 (delta 6), reused 22 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 5.70 KiB | 5.70 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/GPU_mode
CMakelists.txt	matrix_multiplication  test_colab_server.ipynb
include		README.md	       vector_add


# Building all CUDA kernels

In [31]:
# Configure the project (top-level CMakeLists.txt)
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release

# Build all targets (matmul, vec_add, etc.)
!cmake --build build -j 4


CMake Error: The source directory "/content/GPU_mode" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
Error: /content/GPU_mode/build is not a directory


# Optional build single kernels

In [ ]:
# kernels = ["matmul", "vec_add"]  # target names from each CMakeLists.txt

# for k in kernels:
#     print(f"\n=== Building {k} ===")
#     !cmake --build build --target {k} -j 4


# Running all Kernels

In [ ]:
kernels = ["matmul", "vec_add"]  # add more later: "softmax", "conv", etc.

for k in kernels:
    print(f"\n=== Running {k} ===")
    !./build/bin/{k}

# CUDA MemCheck all kernels

In [ ]:
kernels = ["matmul", "vec_add"]

for k in kernels:
    print(f"\n=== cuda-memcheck {k} ===")
    !cuda-memcheck --leak-check full ./build/bin/{k} > profiles/{k}_memcheck.txt 2>&1
    print(f"Saved profiles/{k}_memcheck.txt")


# NCU Profile each kernel all kernels

In [ ]:
from IPython.display import IFrame
import os

kernels = ["matmul", "vec_add"]

os.makedirs("profiles", exist_ok=True)

for k in kernels:
    print(f"\n=== Profiling {k} with ncu ===")
    # HTML + .ncu-rep into profiles/<k>.*
    !ncu -f --set full --export html -o profiles/{k} ./build/bin/{k}

    # Show HTML inline
    display(IFrame(f"profiles/{k}.html", width=1024, height=600))


In [ ]:
# print("Running matmul:")
# !./build/bin/matmul

# print("\nRunning vec_add:")
# !./build/bin/vec_add
